In [1]:
from faker import Faker
import sqlite3
import pandas as pd
import random

In [2]:
fake = Faker()
NUM_AUTHORS = 4

In [10]:
# Create a new database
conn = sqlite3.connect('fake_data.db')
c = conn.cursor()

In [4]:
# Create the blog post table
c.execute('''CREATE TABLE posts(
                id INTEGER PRIMARY KEY,          
                author_id INTEGER,
                published text, 
                title text, 
                body text)
          ''')

# Create the author table
c.execute('''CREATE TABLE authors(
                id INTEGER PRIMARY KEY, 
                name text, 
                email text)
          ''')

In [5]:
# Create our fake authors
authors = []
for i in range(NUM_AUTHORS):
    authors.append(fake.name())

In [6]:
# Write the authors to the database
for i, author in enumerate(authors):
    c.execute('''INSERT INTO authors (id, name, email) 
                  VALUES (?, ?, ?)''', (i, author, fake.email()))

conn.commit()

In [7]:
# Create the fake posts
# Give each author between 5 and 10 posts
# Randomize the date to be within the last year
for i, author in enumerate(authors):
    num_posts = random.randint(5, 10)

    for j in range(num_posts):
        published = fake.date_time_this_year()
        title = fake.sentence()
        body = fake.text()
        
        c.execute('''INSERT INTO posts (author_id, published, title, body) 
                      VALUES (?, ?, ?, ?)''', (i, published, title, body))
        
conn.commit()

In [8]:
# Load up the table in pandas to check it out
df = pd.read_sql_query("SELECT * FROM posts", conn)
df.head()

,id,author_id,published,title,body
0,1,0,2023-05-11 12:22:34,System expert teach somebody identify weight c...,Size four of Republican let. Participant learn...
1,2,0,2023-03-21 02:44:17,Fact prepare offer much.,Opportunity population well that. Six really t...
2,3,0,2023-06-05 03:33:20,Remain music value really often.,Teach word you deep increase.\nSo drug here in...
3,4,0,2023-09-19 19:33:43,Sell you show professor.,Culture discuss ago sister will product rather...
4,5,0,2023-07-17 08:46:26,Difference policy foot as north car total attack.,Discuss yes community someone keep.\nTen towar...


In [11]:
# and the authors table
df = pd.read_sql_query("SELECT * FROM authors", conn)
df.head()

,id,name,email
0,0,Tiffany Fields,courtneybarr@example.net
1,1,Christine Shaw,bakerkelly@example.net
2,2,Stacey Petersen,burkemarc@example.net
3,3,Cheryl Clark,scottsanders@example.com


In [12]:
# Close the connection
conn.close()